In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
raw_data = pd.read_csv('data/raw_data.csv')
ndc_countries = raw_data['country_wb'].to_list()

*Source*: https://rise.worldbank.org/country

In [ ]:
# without headers the website rejects the request (look at your Network Monitor to see which one to use)
HDR = {
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:69.0) Gecko/20100101 Firefox/69.0'
}

def get_country_info(country):
    """Extract relevant info about a country"""
    page = requests.get('https://rise.worldbank.org/country/{}'.format(country), headers=HDR)
    
    soup = BeautifulSoup(page.text, 'html.parser')
    node_number = int(soup.find_all('a', class_='Prt')[0]['href'].split('/')[-1])

    xls_io = requests.get(f'https://rise.worldbank.org/export/excel/{node_number}?country={node_number}', headers=HDR)

    fname = '{}_info.xls'.format(country)
    output = open(fname, 'wb')
    output.write(xls_io.content)
    output.close()
    
    
    return pd.read_excel(fname)

Get list of countries of World Bank database

In [ ]:
page = requests.get('https://rise.worldbank.org/countries', headers=HDR)
soup = BeautifulSoup(page.text, 'html.parser')

countries = soup.find_all('div', class_='country-name')
countries_names = []
for i in range(len(countries)):
    country_name = list(countries[i].children)[0]['href'].split('/')[2]
    print(country_name)
    countries_names.append(country_name)

In [ ]:
raw_data = pd.read_csv('data/raw_data.csv')
ndc_countries = raw_data['country_wb'].to_list()

missing = []
matching = []

for c in ndc_countries:
    if c in countries_names:
        matching.append(c)
    else:
        missing.append(c)
missing



In [ ]:
# Download the info of the matching countries (slow)
for c in matching:
    get_country_info(c)